In [ ]:
using Pkg;Pkg.activate(".") 
using LaTeXStrings, LinearAlgebra, Zygote, ForwardDiff, Printf, Plots
include("utils.jl")

#### 1-d Chain :  

- Particles at positions $X = (x_1, \dots, x_N)$, with periodic boundary conditions 
- Hamiltonian operator  $H_{ij}^{\alpha\beta}$
$$
\begin{aligned}
    {H}_{\ell \ell} = 
        \begin{pmatrix}
            0 & 0 \\
            0 & 2
        \end{pmatrix} 
        %
        \qquad 
        %
        {H}_{\ell k} = 
            \begin{pmatrix}
                f_1(r_{\ell k}) &  f_3(r_{\ell k}) \\
                f_3(r_{\ell k}) &  f_2(r_{\ell k})
            \end{pmatrix}
\end{aligned}
$$
where $f_i(r) = (b_i r + a_i ) e^{-r^2}$, $r$ is the torus distance.

Given the Hamiltonian, we diagonalize it, 
$$
    H v_i = \lambda_i v_i
$$
and then compute the density matrix 
$$
    \rho = \sum_i f(\lambda_i) v_i \otimes v_i
$$
We are interested in 
$$
    \nabla_x (\rho_{11} + \rho_{22}).
$$
This gives us the sensitivity of $\rho$ with respect to particle positions $x_j$. We use this to study a form of near-sightedness in electronic structure models. 

In [ ]:
# This can be extended to more general matrices H
# Here we just construct a hamiltonian with 2 orbitals and fixed GAP = 2.0

function hamiltonian(X::AbstractVector{T}) where {T} 
    N = length(X)
    L = N 
    # parameters 
    p = 0.923; e = exp(1)
    a = [1.5, 0.5, 0.0] * e
    b = [-0.5, 0.2, p] * e
    
    # hamiltonian onsite block
    onsite = [0.0 0.0; 0.0 2.0]
    
    # offsite blocks
    function offsite(x, α, β) 
        if α == β == 1  
            return (b[1]*x+a[1]) * exp(-x^2)
        elseif α == β == 2
            return (b[2]*x+a[2]) * exp(-x^2)
        else      
            return (b[3]*x+a[3]) * exp(-x^2)
        end
    end
        
    # periodic distance 
    perdist(i, j) = sin(π / L * abs(X[i] - X[j])) * L / (π)
    
    function _h(i, j, α,  β)
        x = perdist(i, j) 
        if i == j
            return onsite[α, β] 
        else 
            return offsite(x, α, β)
        end
    end
    
    H = [ _h(i÷2+1, j÷2+1, i%2+1, j%2+1) for i = 0:2*N-1, j = 0:2*N-1 ]
    return H 
            
end

In [ ]:
# a quick little test that the hamiltonian evaluates ok 
# and that the eigendecomposition behaves as expected: 
N = 5
X = (1:N) + 0.03 * (rand(N) .- 0.5)
H = hamiltonian(X)
λ, V = eigen(H)
norm(H * V[:, 1] - λ[1] * V[:, 1])

In [ ]:
# Now we can implement the density matrix
# or rather the trace of the ρ₁₁ block 
# which we want to differentiate. 
function ρ11(X)
    N = length(X) 
    H = hamiltonian(X)
    λ, V = eigen(H)
    β = 10.0 
    fd = 1 ./ (1 .+ exp.(β * λ))
    return sum( (V[1, :].^2 + V[2,:].^2) .* fd )
end

In [ ]:
# clearly Zygote is happy to do this for us!!
N = 6
X = (1:N) + 0.01 * (rand(N) .- 0.5)
Zygote.gradient(ρ11, X)[1]

### Finite difference test

(is the gradient correct?)

In [ ]:
fdtest( ρ11, x -> Zygote.gradient(ρ11, x)[1], X )
println("nice")

### Decay of   $\frac{\rho(\mathcal{H})_{ℓℓ}}{dr_m}$

This was the original problem we wanted to work on.

In [ ]:
N = 100
X = (1:N) .+ rand(N)
g = Zygote.gradient(ρ11, X)[1];

In [ ]:
g

In [ ]:
using Plots

scatter(1:30, abs.(g[1:30]), yscale = :log10, size = (600, 500), 
        xlabel = L"x_{j}", ylabel = L"∇_{x_j} ρ_{11}", 
        label = "empirical decay")
plot!( [10,20], 100*exp.(- [10, 20]), lw=2, ls=:dash, c=:black, 
        label = L"\sim e^{-r}")